# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import statsmodels.stats as smd
import pylab
import statsmodels.stats.weightstats
import statsmodels.stats.proportion
pd.set_option('display.max_columns',500)

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [12]:
# number of callbacks for black-sounding names
print(sum(data[data.race=='w'].call))
#number of callbacks for white-sounding names
print(sum(data[data.race=='b'].call))

235.0
157.0


In [7]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,email,computerskills,specialskills,firstname,sex,race,h,l,call,city,kind,adid,fracblack,fracwhite,lmedhhinc,fracdropout,fraccolp,linc,col,expminreq,schoolreq,eoe,parent_sales,parent_emp,branch_sales,branch_emp,fed,fracblack_empzip,fracwhite_empzip,lmedhhinc_empzip,fracdropout_empzip,fraccolp_empzip,linc_empzip,manager,supervisor,secretary,offsupport,salesrep,retailsales,req,expreq,comreq,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,1,0,0,1,0,Allison,f,w,0.0,1.0,0.0,c,a,384.0,0.989360,0.005500,9.527484,0.274151,0.037662,8.706325,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,6,1,1,1,0,Kristen,f,w,1.0,0.0,0.0,c,a,384.0,0.080736,0.888374,10.408828,0.233687,0.087285,9.532859,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,1,1,0,1,0,Lakisha,f,b,0.0,1.0,0.0,c,a,384.0,0.104301,0.837370,10.466754,0.101335,0.591695,10.540329,1.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,5,0,1,1,1,Latonya,f,b,1.0,0.0,0.0,c,a,384.0,0.336165,0.637370,10.431908,0.108848,0.406576,10.412141,0.0,5,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,5,1,1,1,0,Carrie,f,w,1.0,0.0,0.0,c,a,385.0,0.397595,0.180196,9.876219,0.312873,0.030847,8.728264,0.0,some,,1.0,9.4,143.0,9.4,143.0,0.0,0.204764,0.727046,10.619399,0.070493,0.369903,10.007352,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

<div class="span5 alert alert-success">
<p>


Since the it is a bernoulli distribution as we have a "success"(got a call) or "failure"(didnt get a call) as our random variable for two different populations. Since we are comparing the proportion of the populations will use the test for comparing the difference between the proportions of two populations. The Z test will be used since we have a lot of samples and CLT can be used.


CLT theoreom definitely does apply because we have 4870 random samples which would lead to a normal distribution under the CLT theorem and can be seen visually as well.

The null hypothesis is the difference between the proportions is 0 that the callback is the same for both white sounding names and black sounding names
ie, p1 - p2 = 0

The alternate hypothesis is the difference between the proportions is not 0 
ie, p1 - p2 != 0



</p>
</div>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

In [23]:
# Your solution to Q3 here

w_callback = np.sum(data[data.race=='w'].call)
b_callback = np.sum(data[data.race=='b'].call)

w_callback_p = w_callback/ len(data[data.race=='w'].call)
b_callback_p = b_callback/ len(data[data.race=='b'].call)




0.09650924024640657

In [34]:
def two_proportion_test(p1,size1,p2,size2):
    
    prop1 = p1/size1
    prop2 = p2/size2
    
    prop_pooled = (p1+p2)/(size1+size2)
    
    var = prop_pooled * (1 - prop_pooled) * (1 / size1 + 1 / size2)
    zscore = np.abs(prop2 - prop1) / np.sqrt(var)
    
    p_value = stats.norm.sf(abs(zscore))*2
    
    return zscore,p_value
    

In [37]:
z,p = two_proportion_test(235,2435,157,2435)

In [41]:
print('The Z score is: %f'%z)
print('The p value is: %f'%p)

The Z score is: 4.108412
The p value is: 0.000040


In [43]:
print('Since p value is very low we can reject Ho and say the callback for white sounding men and black sounding men is not the same')

Since p value is very low we can reject Ho and say the callback for white sounding men and black sounding men is not the same


#### Confidence interval

In [64]:
z_crit = 1.96 #two tail test

p_diff = w_callback_p - b_callback_p

std_err = np.sqrt((((w_callback_p)*(1-w_callback_p))/2435) + (((b_callback_p) * (1-b_callback_p))/2435))

ci_high = p_diff + z_crit*(std_err)
print('Higher CI level %f'%ci_high)

ci_low = p_diff - z_crit*(std_err)
print('Lower CI level %f'%ci_low)



Higher CI level 0.047288
Lower CI level 0.016777


<div class="span5 alert alert-success">
<p> Rejecting the null hypothesis shows that the probability of white sounding names to recieve a callback is greater  since the pw-pb > 0. The 95% confidence interval does not contain the the mean of and is more right skewed because white sounding names recieve more calls. The 95% CI is 0.047288 to 0.016777 where the true population mean really lies.

Race/name is a factor that is an important factor for callback however, other factors could also influence callback that can be tested using analysis and multiregression analysis. Various other factors would need to be tested to determine if that is the sole reason for callback.

</p>
</div>